# Dataset

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [4]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [5]:
user_recos[33][0]

RecoPath(nodes=[RecoNode(type='user', entity_id=33), RecoNode(type='product', entity_id=2346), RecoNode(type='user', entity_id=2678), RecoNode(type='product', entity_id=1762)], rels=[RecoRel(in_node=RecoNode(type='user', entity_id=33), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=2346)), RecoRel(in_node=RecoNode(type='user', entity_id=2678), relation='watched', out_node=RecoNode(type='product', entity_id=1762))])

# Explanation

In [6]:
from typing import List

from models.reco.reco_path import RecoPath


def generate_facts(path: RecoPath):
    facts_txt = "% Path: \n"
    for rel in path.rels:
        facts_txt += rel.to_facts() + "\n"
        user = user_registry.find_by_eid(rel.in_node.entity_id)
        product = product_registry.find_by_eid(rel.out_node.entity_id)
        facts_txt += rating_registry.find_user_product_rating(user.uid, product.pid).to_facts() + "\n"
    facts_txt += "% Background Knowledge: \n"
    for node in path.nodes:
        if node.type == "user":
            user = user_registry.find_by_eid(node.entity_id)
            facts_txt += user.to_facts() + "\n"
        elif node.type == "product":
            product = product_registry.find_by_eid(node.entity_id)
            facts_txt += product.to_facts() + "\n"
    return facts_txt
            

In [7]:
print(generate_facts(user_recos[33][0]))

% Path: 
watched(User33, Product2346)
rated(User33, Product2346, 4)
watched(User2678, Product2346)
rated(User2678, Product2346, 4)
watched(User2678, Product1762)
rated(User2678, Product1762, 3)
% Background Knowledge: 
gender(User33, "F")
age(User33, "18-24")
name(Product2346, "Fish Called Wanda, A (1988)")
genre(Product2346, "Comedy")
gender(User2678, "F")
age(User2678, "18-24")
name(Product1762, "Fast Times at Ridgemont High (1982)")
genre(Product1762, "Comedy")



In [8]:
import dotenv

dotenv.load_dotenv()

True

In [9]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [10]:
# # answering_chain.input_schema.schema()
# from langchain_core.prompts import PromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_community.llms import HuggingFaceEndpoint
# from langchain.chains import LLMChain
# from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# from langchain.prompts import PromptTemplate
# import json
# # from langchain_core.llms import OpenAI

# reasoning_prompt_template = """You are an expert in graph-based recommender systems.
# You try to follow the following explanation goal:
# 1. **Transparency:** Clearly explain how the recommendation algorithm made the decision.
# 2. **Scrutability:** Allow the user to provide feedback if the recommendation seems incorrect.
# 3. **Trust:** Build user’s confidence in the recommender system.
# 4. **Effectiveness:** Help user make informed decisions about the recommendation.
# 5. **Efficiency:** Provide a quick explanation to facilitate faster decision-making.
# 6. **Persuasiveness:** Convince user of the relevance of the recommendation.
# 7. **Satisfaction:** Enhance the ease of use and overall experience of the system for the user.
# Given the background knowledge: {background_knowledge},
# explain why the movie {movie} was recommended to the user {user}.
# """

# reasoning_prompt = PromptTemplate.from_template(reasoning_prompt_template)

# reasoning_llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
#     **{
#         "max_new_tokens": 1500,
#         "temperature": 0.1,
#         "top_k": 50,
#         "top_p": 0.9,
#         "repetition_penalty": 1.1,
#         "length_penalty": 0.9,
#         "frequency_penalty": 1.0,
#     },
# )

# answering_llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
#     **{
#         "max_new_tokens": 200,
#         "temperature": 

In [11]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

reasoning_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    **{
        "max_new_tokens": 1500,
        "temperature": 0.1,
        "top_k": 50,
        "top_p": 0.9,
        "repetition_penalty": 1.1
    },
)

answering_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    **{
        "max_new_tokens": 200,
        "temperature": 0.4,
        "top_k": 30,
        "top_p": 0.8,
        "repetition_penalty": 1.05
    },
)

reasoning_template = """You are an expert in graph-based recommender systems.
You try to follow the following explanation goal:
1. **Transparency:** Clearly explain how the recommendation algorithm made the decision.
2. **Scrutability:** Allow the user to provide feedback if the recommendation seems incorrect.
3. **Trust:** Build user’s confidence in the recommender system.
4. **Effectiveness:** Help user make informed decisions about the recommendation.
5. **Efficiency:** Provide a quick explanation to facilitate faster decision-making.
6. **Persuasiveness:** Convince user of the relevance of the recommendation.
7. **Satisfaction:** Enhance the ease of use and overall experience of the system for the user.
Given the background knowledge: {background_knowledge},
explain why the movie "{product_name}" was recommended to the user {user}.
"""

reasoning_prompt = PromptTemplate.from_template(reasoning_template)

answer_template = """Based on the reasoning provided: {reasoning},
give a concise and helpful response about why the movie "{product_name}" was recommended to the user {user} 
without repeating the explanation goals.
"""
answer_prompt = PromptTemplate.from_template(answer_template)


/home/balfroim/.cache/pypoetry/virtualenvs/trail24-UZVMavSK-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/balfroim/.cache/huggingface/token
Login successful


In [12]:
reasoning_chain = reasoning_prompt | reasoning_llm
answering_chain = answer_prompt | answering_llm

In [13]:
from recommendation.explainers.cot_explainer import COTExplainer

cot_explainer = COTExplainer(product_registry, user_registry, rating_registry, reasoning_chain, answering_chain)

In [23]:
path = user_recos[33][0]
continuation, trace = cot_explainer.explain(path)

In [25]:
print("reasoning prompt:")
print(trace.reasoning_trace.prompt)
print("reasoning completion:")
print(trace.reasoning_trace.completion)
print("answer prompt:")
print(trace.answering_trace.prompt)
print("answer completion:")
print(trace.answering_trace.completion)

reasoning prompt:
You are an expert in graph-based recommender systems.
You try to follow the following explanation goal:
1. **Transparency:** Clearly explain how the recommendation algorithm made the decision.
2. **Scrutability:** Allow the user to provide feedback if the recommendation seems incorrect.
3. **Trust:** Build user’s confidence in the recommender system.
4. **Effectiveness:** Help user make informed decisions about the recommendation.
5. **Efficiency:** Provide a quick explanation to facilitate faster decision-making.
6. **Persuasiveness:** Convince user of the relevance of the recommendation.
7. **Satisfaction:** Enhance the ease of use and overall experience of the system for the user.
Given the background knowledge: % Path: 
watched(User33, Product2346)
rated(User33, Product2346, 4)
watched(User2678, Product2346)
rated(User2678, Product2346, 4)
watched(User2678, Product1762)
rated(User2678, Product1762, 3)
% Background Knowledge: 
gender(User33, "F")
age(User33, "18-24

In [15]:
# # for i, path in enumerate(user_recos[33][0]):
# # path = user_recos[0][0]
# def explain(path):
#     bk = generate_facts(path)
#     product_eid = path.recommendation[1].entity_id
#     product = product_registry.find_by_eid(product_eid)
#     user_eid = path.recommendation[0].entity_id
#     user = user_registry.find_by_eid(user_eid)
#     result =   cot_chain.invoke(
#         {"background_knowledge": bk, "product_name": product.name, "user": str(user)},
#     )
#     print("-----------------")
#     print(result)
#     # print(result)
#     # with open(f"explanations/exp_{i}.txt", "w") as f:
#     #     f.write(llm_chain.run(explain(reco)))

In [16]:
# path = user_recos[33][0]
# explain(path)
# # await explain(path)

In [17]:
# # import logging
# # logging.basicConfig(filename='./error.log', level=logging.DEBUG, 
# #                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# # logger=logging.getLogger(__name__)

# for user_reco in user_recos.values():
#     for reco in user_reco:
#         # try:
#         #     # raise Exception("test")
#         #     explain(reco)
#         # except Exception as e:
#         #     print(e)
#         #     logger.error(e)


# # FIXME: some generate exception
#     # print(user_reco)
# # for path in user_recos[0]:
#     # explain(path)
#     # break